In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from google.colab import files

In [ ]:
from PIL import Image
import os
image_dir = '/content/drive/MyDrive/brain tumor/yes'
image_files = os.listdir(image_dir)

yes = []

for file in image_files:
    image_path = os.path.join(image_dir, file)
    image = Image.open(image_path)
    yes.append(image)
yes[0].show()

In [ ]:
from PIL import Image
import os


image_dir = '/content/drive/MyDrive/brain tumor/no'


image_files = os.listdir(image_dir)

no = []

for file in image_files:
    image_path = os.path.join(image_dir, file)
    image = Image.open(image_path)
    no.append(image)

no[0].show()


In [ ]:
#now we have stored the yes and no in form of a list
#like yes[], no[] now we see properties
print(len(yes))
print(len(no))

In [ ]:
yes_train=yes[:108]
yes_test=yes[108:]
no_train=no[:69]
no_test=no[69:]

In [ ]:
#one hot vector encoding
from keras.utils import to_categorical
yes_train_labels=[1]*108
no_train_labels=[0]*69
yes_test_labels=[1]*47
no_test_labels=[0]*29

In [ ]:
#merging train labels and train pics into one dataset
#namely train and other namely test
train_labels=[]
train_labels.extend(yes_train_labels)
train_labels.extend(no_train_labels)
test_labels=[]
test_labels.extend(yes_test_labels)
test_labels.extend(no_test_labels)

In [ ]:
def resize_images(images,x,y):
  resized_images=[]
  for image in images:
    new_image = image.resize((x, y))
    resized_images.append(new_image)
  return resized_images

In [ ]:
train_images=[]
train_images.extend(yes_train)
train_images.extend(no_train)
test_images=[]
test_images.extend(yes_test)
test_images.extend(no_test)
train_images=resize_images(train_images,100,100)
test_images=resize_images(test_images,100,100)

In [ ]:
height, width = train_images[0].size

# Print the dimensions
print("Width:", width)
print("Height:", height)
print("Channels:", train_images[0].getbands())


height, width = test_images[0].size

# Print the dimensions
print("Width:", width)
print("Height:", height)
print("Channels:", test_images[0].getbands())

In [ ]:
#the output tells that there are three channels
#RGB is this indication

The output tells us that there are in total 3 channels RGB
Now we have preproccessed out data to meet the criteria
Now it is fit to train
Now main question is howcome the blck and white grayscale images are showing this RGB?

In [ ]:
#finally we have preproccessed our data
#now our task is to implement a model
# I will first import all the needed packages then start
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

In [ ]:
resized_train_images = [img.resize((100, 100)).convert('RGB') for img in train_images]
resized_test_images = [img.resize((100, 100)).convert('RGB') for img in test_images]

# Convert resized images to NumPy arrays
train_images = np.asarray([np.array(img) for img in resized_train_images])
test_images = np.asarray([np.array(img) for img in resized_test_images])

# Convert data type and normalize pixel values
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [ ]:
print("Resized train image shapes:")
for img in resized_train_images:
    print(np.array(img).shape)

print("Resized test image shapes:")
for img in resized_test_images:
    print(np.array(img).shape)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load the pre-trained VGG16 model without the top classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the feature extraction layers
for layer in base_model.layers:
    layer.trainable = False

# Create your old model or load it from a saved file
# Replace this with your own model architecture and weights
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='sigmoid')
    # Add layers from your old model here
])

In [ ]:
#compiling the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#now we fir the model
model.fit(train_images, to_categorical(train_labels), batch_size=32, epochs=20, validation_data=(test_images, to_categorical(test_labels)))